In [6]:
from transformers import pipeline
import torch
from transformers import DetrForObjectDetection, DetrImageProcessor, pipeline
import torchvision.transforms as transforms
import cv2 as cv
from PIL import Image
from transformers import AutoProcessor, AutoModelForSpeechSeq2Seq
import gradio as gr 
import requests
import numpy as np
from transformers import BlenderbotTokenizer, BlenderbotForConditionalGeneration

In [2]:
model_path = "./models/object-detection-resnet-50"
processor =  DetrImageProcessor.from_pretrained(model_path)
model = DetrForObjectDetection.from_pretrained(model_path)

In [4]:
cap = cv.VideoCapture(0)
while True:    
    ret, frame = cap.read()
    if not ret:
        break
    frame_tensor = torch.as_tensor(frame, dtype=torch.float32)
    image_frame = processor(images=frame_tensor, return_tensors="pt")
    outputs= model(**image_frame)
    
    target_sizes = torch.tensor([[frame_tensor.shape[1], frame_tensor.shape[2]]])

    detections = processor.post_process_object_detection(outputs, target_sizes=target_sizes, threshold=0.9)[0]
    for score, label, box in zip(detections["scores"], detections["labels"], detections["boxes"]):
        box = [int(i) for i in box.tolist()]
        cv.rectangle(frame, (box[0], box[1]), (box[2], box[3]), (0, 255, 0), 2) 
        label_text = f"{model.config.id2label[label.item()]}: {round(score.item(), 2)}"
        cv.putText(frame, label_text, (box[0], box[1] - 10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)
    if label_text.startswith ("person"):
        start_conversation()
        break
    cv.imshow("frame", frame)
    if cv.waitKey(1) == ord('q'):
         break
cap.release()
cv.destroyAllWindows()


NameError: name 'start_conversation' is not defined

#### creating a conversation with someone
Reading the input from the microphone
detect the spoken langauge 
convert the microphone sampling rate to 16kz
convert the spoken words to text

In [21]:
def start_conversation():
    transcribed_text = ""
    
    whisper_model_path= "./models/automatic-speech-recognition"
    processor = AutoProcessor.from_pretrained(whisper_model_path)
    model = AutoModelForSpeechSeq2Seq.from_pretrained(whisper_model_path, use_safetensors=True)

    torch_dtype = torch.float32

    whisper_pipe = pipeline(
    "automatic-speech-recognition",
    model=model,
    tokenizer=processor.tokenizer,
    feature_extractor=processor.feature_extractor,
    max_new_tokens=128,
    chunk_length_s=15,
    batch_size=16,
    torch_dtype=torch_dtype
    )
    demo = gr.Blocks()

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [14]:
def transcribe_speech(file_path):
    if file_path is None:
        gr.Warning("No audio found please try again")
        return ""
    output = whisper_pipe(file_path)
    transcribed_text = output[text]
    return output["text"]

In [15]:
mic_transcribe = gr.Interface(
    fn =transcribe_speech,
    inputs = gr.Audio(sources="microphone",
                      type="filepath"),
    outputs = gr.Textbox(label="Transcription",
                         lines=3),
    allow_flagging="never")

In [20]:
with demo:
    gr.TabbedInterface(
        [mic_transcribe],
        ["Transcribe Microphone"]
    )
    demo.launch(debug=True, share=True)

DuplicateBlockError: At least one block in this Blocks has already been rendered.

##### creating conversation context using blenderbot model
from the transcribed spoken words create a conversation chat with
create a context to follow up the text messages converted and spoken

In [ ]:
def creating_conversation_with_blenderbot(transcribed_text):
    from transformers import Conversation
    blender_model_path = "./models/facebook"
    tokenizer = BlenderbotTokenizer.from_pretrained(blender_model_path)
    model = BlenderbotForConditionalGeneration.from_pretrained(blender_model_path)
    
    blender_bot_pipeline = pipeline(task="conversational", model =model)
    
    conversation = Conversation(transcribed_text)
    conversation = blender_bot_pipeline(conversation)

In [ ]:
def produce_reply_sound(text):
    import pyttsx3
    engine = pyttsx3.init()
    rate = engine.getProperty('rate')
    engine.setProperty('rate', rate- 50)
    engine.say(text)
    engine.runAndWait()

Exception: Impossible to guess which tokenizer to use. Please provide a PreTrainedTokenizer class or a path/identifier to a pretrained tokenizer.